In [1]:
# KBO 일정 크롤링 - KBO 홈페이지
## 1.1 2024 UPDATE - 기존에 사용하던 requests 방식 사용

### 1. 패키지 세팅 
import math
import re
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup as bs
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve

import html5lib

from sklearn.svm import SVC

In [2]:
data = {
    "leId": '1',
    "srIdList": '0,9,6',
    "seasonId": '2024',
    "gameMonth": "05",
    "teamId": ""
}
r = requests.post("https://www.koreabaseball.com/ws/Schedule.asmx/GetScheduleList", data=data)
r

<Response [200]>

In [27]:
root = json.loads(r.content.decode("utf-8"))
print(root['rows'])



[{'row': [{'Text': '05.01(수)', 'Class': 'day', 'Scope': None, 'RowSpan': '5', 'ColSpan': None, 'Width': None, 'TypeObj': None}, {'Text': '<b>18:30</b>', 'Class': 'time', 'Scope': None, 'RowSpan': None, 'ColSpan': None, 'Width': None, 'TypeObj': None}, {'Text': '<span>삼성</span><em><span class="win">9</span><span>vs</span><span class="lose">2</span></em><span>두산</span>', 'Class': 'play', 'Scope': None, 'RowSpan': None, 'ColSpan': None, 'Width': None, 'TypeObj': None}, {'Text': "<a href='/Schedule/GameCenter/Main.aspx?gameDate=20240501&gameId=20240501SSOB0&section=REVIEW' class='btn2 mr5' id='btnReview'>리뷰</a>", 'Class': 'relay', 'Scope': None, 'RowSpan': None, 'ColSpan': None, 'Width': None, 'TypeObj': None}, {'Text': "<a href='/Schedule/GameCenter/Main.aspx?gameDate=20240501&gameId=20240501SSOB0&section=HIGHLIGHT' class='btn2' id='btnHighlight' style='width:65px;'>하이라이트</a>", 'Class': None, 'Scope': None, 'RowSpan': None, 'ColSpan': None, 'Width': None, 'TypeObj': None}, {'Text': 'MS-T'

In [52]:
formedData = [];

for row in root['rows']:
    data = {}
    if row['row'][0]['Class'] == 'day':
        data['date'] = row['row'][0]['Text']
        data['time'] = row['row'][1]['Text']
        formedData.append(data)
    else:
        data['date'] = formedData[-1]['date']
        data['time'] = row['row'][0]['Text']
        formedData.append(data)
    
print(formedData)
# ' '.join([row[0]['Text'],row[1]['Text']])


[{'date': '05.01(수)', 'time': '<b>18:30</b>'}, {'date': '05.01(수)', 'time': '<b>18:30</b>'}, {'date': '05.01(수)', 'time': '<b>18:30</b>'}, {'date': '05.01(수)', 'time': '<b>18:30</b>'}, {'date': '05.01(수)', 'time': '<b>18:30</b>'}, {'date': '05.02(목)', 'time': '<b>18:30</b>'}, {'date': '05.02(목)', 'time': '<b>18:30</b>'}, {'date': '05.02(목)', 'time': '<b>18:30</b>'}, {'date': '05.02(목)', 'time': '<b>18:30</b>'}, {'date': '05.02(목)', 'time': '<b>18:30</b>'}, {'date': '05.03(금)', 'time': '<b>18:30</b>'}, {'date': '05.03(금)', 'time': '<b>18:30</b>'}, {'date': '05.03(금)', 'time': '<b>18:30</b>'}, {'date': '05.03(금)', 'time': '<b>18:30</b>'}, {'date': '05.03(금)', 'time': '<b>18:30</b>'}, {'date': '05.04(토)', 'time': '<b>14:00</b>'}, {'date': '05.04(토)', 'time': '<b>17:00</b>'}, {'date': '05.04(토)', 'time': '<b>17:00</b>'}, {'date': '05.04(토)', 'time': '<b>17:00</b>'}, {'date': '05.04(토)', 'time': '<b>17:00</b>'}, {'date': '05.05(일)', 'time': '<b>14:00</b>'}, {'date': '05.05(일)', 'time': '<b>

In [33]:
root['rows'][0]['row'][0]['Text']

'05.01(수)'

In [ ]:
# 0~1 -> 날짜
# 2 -> 경기 정보 (홈팀 / 홈팀점수 / 원정팀 점수 / 원정팀)
# 7 -> 경기장
# 8 -> 비고 (특수경기)